# Retention Productivity in Article and Article Talk

This notebook is similar to the one fitting a model for edits in Article and Article Talk across the entire 15-day period, but looks only at the second two-week part of that time period (also known as the "retention" period).

In [1]:
# https://stackoverflow.com/a/35018739/1091835
library(IRdisplay)

display_html(
'<script>  
code_show=true; 
function code_toggle() {
  if (code_show){
    $(\'div.input\').hide();
  } else {
    $(\'div.input\').show();
  }
  code_show = !code_show
}  
$( document ).ready(code_toggle);
</script>
  <form action="javascript:code_toggle()">
    <input type="submit" value="Click here to toggle on/off the raw code.">
 </form>'
)

In [2]:
library(data.table)
library(ggplot2)

library(brms) # install.packages("brms")
library(loo) # install.packages("loo")
options(mc.cores = 4)
library(rstanarm) # install.packages("rstanarm")

Loading required package: Rcpp

Loading 'brms' package (version 2.13.5). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').


Attaching package: ‘brms’


The following object is masked from ‘package:stats’:

    ar


This is loo version 2.3.1

- Online documentation and vignettes at mc-stan.org/loo

- As of v2.0.0 loo defaults to 1 core but we recommend using as many as possible. Use the 'cores' argument or set options(mc.cores = NUM_CORES) for an entire session. 

rstanarm (Version 2.19.3, packaged: 2020-02-11 05:16:41 UTC)

- Do not expect the default priors to remain the same in future rstanarm versions.

Thus, R scripts should specify priors explicitly, even if they are just the defaults.

- For execution on a local, multicore CPU with excess RAM we recommend calling

options(mc.cores = parallel::detectCores())

- bayesplot theme set to bayesplot::theme_default()

   * Does _not_ affec

## Configuration variables

In [3]:
## Set BLAS threads to 4 so glmer and loo don't use all cores
library(RhpcBLASctl)
blas_set_num_threads(1)

## parallelization
options(mc.cores = 4)

### Data import and setup

In [4]:
user_edit_data = fread('/home/nettrom/src/Growth-homepage-2019/datasets/newcomer_tasks_edit_data_may2020.tsv',
                       colClasses = c(wiki_db = 'factor'))

In [5]:
## Configuration variables for this experiment.
## Start timestamp is from https://phabricator.wikimedia.org/T227728#5680453
## End timestamp is from the data gathering notebook
start_ts = as.POSIXct('2019-11-21 00:24:32', tz = 'UTC')
end_ts = as.POSIXct('2020-04-9 00:00', tz = 'UTC')

## Start of the Variant A/B test
variant_test_ts = as.POSIXct('2019-12-13 00:32:04', tz = 'UTC')

## Convert user_registration into a timestamp
user_edit_data[, user_reg_ts := as.POSIXct(user_registration_timestamp,
                                           format = '%Y-%m-%d %H:%M:%S.0', tz = 'UTC')]

## Calculate time since start of experiment in weeks
user_edit_data[, exp_days := 0]
user_edit_data[, exp_days := difftime(user_reg_ts, start_ts, units = 'days')]
user_edit_data[exp_days < 0, exp_days := 0]
user_edit_data[, ln_exp_days := log(1 + as.numeric(exp_days))]
user_edit_data[, ln_exp_weeks := log(1 + as.numeric(exp_days)/7)]

## Calculate time since the start of the variant test, again in days and weeks.
## This enables us to do an interrupted time-series model for that.
user_edit_data[, variant_exp_days := 0]
user_edit_data[, variant_exp_days := difftime(user_reg_ts, variant_test_ts, units = 'days')]
user_edit_data[variant_exp_days < 0, variant_exp_days := 0]
user_edit_data[, ln_var_exp_days := log(1 + as.numeric(variant_exp_days))]
user_edit_data[, ln_var_exp_weeks := log(1 + as.numeric(variant_exp_days)/7)]
user_edit_data[, in_var_exp := 0]
user_edit_data[user_reg_ts > variant_test_ts, in_var_exp := 1]

## Convert all NAs to 0, from
## https://stackoverflow.com/questions/7235657/fastest-way-to-replace-nas-in-a-large-data-table
na_to_zero = function(DT) {
  # or by number (slightly faster than by name) :
  for (j in seq_len(ncol(DT)))
    set(DT,which(is.na(DT[[j]])),j,0)
}

na_to_zero(user_edit_data)

## Turn "reg_on_mobile" into a factor for more meaningful plots
user_edit_data[, platform := 'desktop']
user_edit_data[reg_on_mobile == 1, platform := 'mobile']
user_edit_data[, platform := factor(platform)]

## Control variables for various forms of activation
user_edit_data[, is_activated_article := num_article_edits_24hrs > 0]
user_edit_data[, is_activated_other := num_other_edits_24hrs > 0]
user_edit_data[, is_activated := is_activated_article | is_activated_other]

## Control variables for constructive forms of activation
user_edit_data[, is_const_activated_article := (num_article_edits_24hrs - num_article_reverts_24hrs) > 0]
user_edit_data[, is_const_activated_other := (num_other_edits_24hrs - num_other_reverts_24hrs) > 0]
user_edit_data[, is_const_activated := is_const_activated_article | is_const_activated_other]

## Control variables for the number of edits made
user_edit_data[, log_num_article_edits_24hrs := log(1 + num_article_edits_24hrs)]
user_edit_data[, log_num_other_edits_24hrs := log(1 + num_other_edits_24hrs)]
user_edit_data[, log_num_edits_24hrs := log(1 + num_article_edits_24hrs + num_other_edits_24hrs)]

## Control variables for the constructive number of edits made
user_edit_data[, log_num_const_article_edits_24hrs := log(
    1 + num_article_edits_24hrs - num_article_reverts_24hrs)]
user_edit_data[, log_num_const_other_edits_24hrs := log(
    1 + num_other_edits_24hrs - num_other_reverts_24hrs)]
user_edit_data[, log_num_const_edits_24hrs := log(
    1 + num_article_edits_24hrs + num_other_edits_24hrs -
    num_article_reverts_24hrs - num_other_reverts_24hrs)]

## Retention variables
user_edit_data[, is_const_retained_article := is_activated_article &
               ((num_article_edits_2w - num_article_reverts_2w) > 0)]
user_edit_data[, is_const_retained_other := is_const_activated_other &
               ((num_other_edits_2w - num_other_reverts_2w) > 0)]
user_edit_data[, is_const_retained := is_const_activated &
               ((num_article_edits_2w + num_other_edits_2w -
                 num_article_reverts_2w - num_other_reverts_2w) > 0)]

## Variables for number of edits (overall and only constructive)
## across the entire period.
user_edit_data[, num_total_edits_24hrs := num_article_edits_24hrs + num_other_edits_24hrs]
user_edit_data[, num_total_edits_2w := num_article_edits_2w + num_other_edits_2w]
user_edit_data[, num_total_edits := num_total_edits_24hrs + num_total_edits_2w]

user_edit_data[, num_total_const_edits_24hrs := (num_article_edits_24hrs + num_other_edits_24hrs -
                                                 num_article_reverts_24hrs - num_other_reverts_24hrs)]
user_edit_data[, num_total_const_edits_2w := (num_article_edits_2w + num_other_edits_2w -
                                              num_article_reverts_2w - num_other_reverts_2w)]
user_edit_data[, num_total_const_edits := num_total_const_edits_24hrs + num_total_const_edits_2w]

## Variables for number of article edits across the entire period.
user_edit_data[, num_total_article_edits := num_article_edits_24hrs + num_article_edits_2w]

## Priors

In [6]:
## Note that using a student_t distribution for the prior is beneficial because that
## distribution handles outliers better than a Normal.
## See https://jrnold.github.io/bayesian_notes/robust-regression.html
## Thanks to Mikhail for sending that to me!
priors <- prior(cauchy(0, 2), class = sd) +
  prior(student_t(5, 0, 10), class = b)

## Edits during the two-week period

We base this model on the same one used across all namespaces, meaning that we don't expect group-level variation in the effect of mobile. This is mainly because we have few wikis in our dataset, thus we don't expect that to contain meaningful information.

Secondly, based on how we found article *activation* to be the best predictor for article *retention* in the logistic regression models for that, we use number of article edits in the activation period as a predictor in this model. We log-transform it so that it's not a skewed variable.

In [ ]:
article_edits_2w.zinb.mod.2 <- brm(
  bf(num_article_edits_2w ~ is_treatment + reg_on_mobile + log_num_article_edits_24hrs +
     (1 | wiki_db),
     zi ~ wiki_db + reg_on_mobile),
    data = user_edit_data,
    family = zero_inflated_negbinomial(),
    prior = priors,
    iter = 800,
    control = list(adapt_delta = 0.999,
                 max_treedepth = 15)
)

Compiling Stan program...

Start sampling



In [ ]:
## Save the model
save(article_edits_2w.zinb.mod.2, file='models/article_edits_2w.zinb.mod.2.Robj')

In [9]:
summary(article_edits_2w.zinb.mod.2)

 Family: zero_inflated_negbinomial 
  Links: mu = log; shape = identity; zi = logit 
Formula: num_article_edits_2w ~ is_treatment + reg_on_mobile + log_num_article_edits_24hrs + (1 | wiki_db) 
         zi ~ wiki_db + reg_on_mobile
   Data: user_edit_data (Number of observations: 97755) 
Samples: 4 chains, each with iter = 800; warmup = 400; thin = 1;
         total post-warmup samples = 1600

Group-Level Effects: 
~wiki_db (Number of levels: 4) 
              Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS Tail_ESS
sd(Intercept)     0.56      0.45     0.19     1.75 1.00      471      649

Population-Level Effects: 
                            Estimate Est.Error l-95% CI u-95% CI Rhat Bulk_ESS
Intercept                      -1.44      0.32    -2.09    -0.78 1.00      535
zi_Intercept                   -1.30      0.19    -1.70    -0.95 1.00     1054
is_treatment                    0.01      0.05    -0.09     0.11 1.00     1736
reg_on_mobile                  -0.28      0.05    -0.38   

In [14]:
posterior_summary(article_edits_2w.zinb.mod.2)

,Estimate,Est.Error,Q2.5,Q97.5
b_Intercept,-1.436078e+00,3.204774e-01,-2.088476e+00,-7.762674e-01
b_zi_Intercept,-1.303115e+00,1.901128e-01,-1.697839e+00,-9.528199e-01
b_is_treatment,1.320592e-02,5.189356e-02,-8.800105e-02,1.140250e-01
b_reg_on_mobile,-2.770289e-01,5.460994e-02,-3.823261e-01,-1.728011e-01
b_log_num_article_edits_24hrs,1.485884e+00,3.779474e-02,1.411093e+00,1.564141e+00
b_zi_wiki_dbcswiki,-1.571778e+01,1.153371e+01,-4.608092e+01,-3.637621e+00
b_zi_wiki_dbkowiki,-1.113959e+01,8.320500e+00,-3.380748e+01,-2.475501e+00
b_zi_wiki_dbviwiki,5.702457e-01,1.452561e-01,2.863031e-01,8.519385e-01
b_zi_reg_on_mobile,7.077178e-01,1.525309e-01,4.150264e-01,1.023025e+00
sd_wiki_db__Intercept,5.561266e-01,4.467574e-01,1.934580e-01,1.747985e+00


In [12]:
draws <- posterior_samples(article_edits_2w.zinb.mod.2)

In [15]:
with(draws, {
    HP <- plogis(b_Intercept + b_is_treatment)
    Control <- plogis(b_Intercept)
    Diff = HP - Control
    c(
        c(HP = median(HP), Control = median(Control)), # group averages
        quantile(Diff, c(0.025, 0.5, 0.975)) # difference of HP - Control
    )
})

HP     Control        2.5%         50%       97.5% 
 0.19334499  0.19068012 -0.01488819  0.00203369  0.01800820